# PowerGIM Example:
## Offshore grid optimisation (deterministic without uncertain paramters)

This example considers grid connection options for wind farms in the Doggerbnk area (North Sea), including potential trade links to Norway and Germany

---
### Cost model
Investment costs:  
$cost = B + B_{dp}\cdot d\cdot p + B_d\cdot d + \sum_{i\in end}\left(C^{i}_p\cdot p + C^{i}\right)$
* $d$ = distance (km)
* $p$ = power rating (MW)
* $B$ = fixed cost (EUR)
* $B_{dp}$ = cost dependence on both distance and rating (EUR/km/MW)
* $B_d$ = cost dependence on distance (EUR/km)
* $C$ = fixed endpoint cost (L=on land, S=at sea) (EUR)
* $C_p$ = endpoint cost dependence on rating (EUR/MW)

Operational costs:  
$cost = P_{g} \cdot mc_{g}$
* $P_{g}$ = generator output (MW)
* $mc_{g}$ = generator marginal cost (EUR/MW)

In [20]:
%matplotlib inline
import powergama
import powergama.powergim as pgim
import powergama.plots
import powergama.sampling
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import numpy.random as rnd
import copy
import lxml.etree
import yaml
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#specify random number seed so re-running the script gives the same result
rnd.seed(2016) 
reuse_sample = True

### Reading data and setting up optimisation model

In [21]:
grid_data = powergama.GridData()
grid_data.readSipData(nodes = "data/dog_nodes.csv",
                  branches = "data/dog_branches.csv",
                  generators = "data/dog_generators.csv",
                  consumers = "data/dog_consumers.csv")
file_parameters = 'data/dog_data_irpwind.yaml'

# Profiles:
if reuse_sample:
    print("\n<> Loading time-series sample...")
    samplesize = 100
    grid_data.readProfileData(filename= "data/timeseries_sample_100_rnd2016.csv",
                              timerange=range(samplesize), timedelta=1.0)
else:
    # create new sample
    grid_data.readProfileData(filename = "data/timeseries_doggerbank.csv",
                              timerange = range(8760), timedelta = 1.0)

    print("\n<> Sampling...\n")
    samplingmethod = 'kmeans'
    samplesize = 3
    profiledata_sample = powergama.sampling.sampleProfileData(data=grid_data,
        samplesize=samplesize,sampling_method=samplingmethod)
    #profiledata_sample.to_csv("data/timeseries_sample_100_rnd2016.csv")
    grid_data.timerange = range(profiledata_sample.shape[0])
    grid_data.profiles = profiledata_sample



<> Loading time-series sample...


In [49]:
displayData = False
num_lines = 30
if displayData:
    with open(file_parameters, 'r') as stream:
        data = yaml.safe_load(stream)
        data = data['powergim']

    print("PARAMETERS - nodetype:")
    display(pd.DataFrame(data['nodetype']).T)
    print("PARAMETERS - branchtype:")
    display(pd.DataFrame(data['branchtype']).T)
    print("PARAMETERS - gentype:")
    display(pd.DataFrame(data['gentype']).T)
    print("PARAMETERS - parameters:")
    display(pd.DataFrame({'value':data['parameters']}))
    print("\nNodes:")
    display(grid_data.node.head(num_lines))
    print("Branches:")
    display(grid_data.branch.head(num_lines))
    print("Consumers:")
    display(grid_data.consumer.head(num_lines))
    print("Generators:")
    display(grid_data.generator.head(num_lines))
    print("Profiles:")
    display(grid_data.profiles.head(num_lines))

In [470]:
grid_data.profiles.min()

Unnamed: 0             0.000000
const                  1.000000
wind0.3                0.039958
wind1.2                0.020603
prices_GB             32.125343
prices_DE           -254.000000
prices_NO_south        9.934286
demand_GB          22050.080460
demand_DE          30511.500000
demand_NO           9528.954023
dtype: float64

### Plotting input data

In [51]:
# A little trick to use powergama map plot 
# (need to split branches into ac and dc)
plot_data = copy.deepcopy(grid_data)
plot_data.spreadNodeCoordinates(inplace=True,radius=0.04)
plot_data.branch = plot_data.branch[plot_data.branch['capacity']>0]
mask_dc = plot_data.branch['type'].str.startswith('dc')
plot_data.dcbranch = plot_data.branch[mask_dc]
plot_data.branch = plot_data.branch[~mask_dc]

m=powergama.plots.plotMap(pg_data=plot_data,pg_res=None,
    filename=None,branchtype="capacity",nodetype="area")
print("\nExisting capacity:")
m


Existing capacity:


In [52]:
# A little trick to use powergama map plot 
# (need to split branches into ac and dc)
plot_data = copy.deepcopy(grid_data)
plot_data.spreadNodeCoordinates(inplace=True,radius=0.04)
mask_dc = plot_data.branch['type'].str.startswith('dc')
plot_data.dcbranch = plot_data.branch[mask_dc]
plot_data.branch = plot_data.branch[~mask_dc]

m=powergama.plots.plotMap(pg_data=plot_data,pg_res=None,
    filename=None,branchtype="capacity",nodetype="area")
print("\nTransmission corridors considered in optimisation:")
m


Transmission corridors considered in optimisation:


### Creating and solving MILP problem

* Choose solver - CBC is freely available, but Gurobi is faster
* Solving takes some time, using CBC on a normal latptop takes about 30 minutes

In [58]:
sip = pgim.SipModel()
dict_data = sip.createModelData(grid_data,
    datafile=file_parameters,
    maxNewBranchNum=5,maxNewBranchCap=5000)
model = sip.createConcreteModel(dict_data) 

#Enable access to dual values
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Reading from yaml
Computing B and DA matrices...
Creating B and DA coefficients...


In [60]:
#opt = pyo.SolverFactory('gurobi',solver_io='python')
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, 
                    tee=True, #stream the solver output
                    keepfiles=False, #print the LP file for examination
                    symbolic_solver_labels=True) # use human readable names
print("=====================================",results['Solver'][0])

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Oct 29 2020 

command line - c:\users\hsven\bin\cbc.exe -printingOptions all -import C:\Users\hsven\AppData\Local\Temp\tmpcr_0vbfd.pyomo.lp -stat=1 -solve -solu C:\Users\hsven\AppData\Local\Temp\tmpcr_0vbfd.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 15347 (-6582) rows, 12882 (-4727) columns and 47124 (-25248) elements
Statistics for presolved model
Original problem has 83 integers (50 of which binary)
Presolved problem has 51 integers (18 of which binary)
==== 12296 zero objective 555 different
==== absolute objective values 553 different
==== for integers 0 zero objective 27 different
==== for integers absolute objective values 27 different
===== end objective counts


Problem has 15347 rows, 12882 columns (586 with objective) and 47124 elements
There are 522 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 12864 of type 0.0->up, 0 of type lo->inf, 
0 of type

ApplicationError: Solver (cbc) did not exit normally

### Analyse results (plot and save to file)

In [543]:
# Export results to XLSX file:
# sip.saveDeterministicResults(model=model,excel_file='res_det_result.xlsx')

In [520]:
def myplot(grid):
    plot_data = copy.deepcopy(grid)
    plot_data.spreadNodeCoordinates(inplace=True,radius=0.04)
    mask_dc = ~plot_data.branch['type'].str.startswith('ac')
    plot_data.dcbranch = plot_data.branch[mask_dc]
    plot_data.branch = plot_data.branch[~mask_dc]
    m = powergama.plots.plotMap(pg_data=plot_data,pg_res=None,
        filename=None,branchtype="capacity",nodetype="area")
    return m
    
print("INPUT:")
m_input = myplot(grid_data)
display(m_input)

print("STAGE 1:")
grid_res = sip.extractResultingGridData(grid_data,model=model)
m_res1 = myplot(grid_res)
display(m_res1)

print("STAGE 2:")
grid_res2 = sip.extractResultingGridData(grid_data,model=model,stage=2)
m_res2 = myplot(grid_res2)
display(m_res2)

INPUT:
Nodes...
AC branches...
DC branches...
Consumers...
Generators...


STAGE 1:
Nodes...
AC branches...
DC branches...
Consumers...
Generators...


STAGE 2:
Nodes...
AC branches...
DC branches...
Consumers...
Generators...


In [521]:
# Export plots to HTML:
# m_res1.save("res_det_stage1.html")

In [534]:
# Show a previously saved html image:
# import IPython
# IPython.display.IFrame("res_det_stage1.html",width='60%',height="400")